# 95. CNNを超えて：Vision TransformerとMLP-Mixer

## 学習目標

このノートブックでは、以下を学びます：

1. **Vision Transformer (ViT)**の基本概念
2. **Self-Attention**による大域的な依存関係のモデリング
3. **MLP-Mixer**：MLPだけで画像認識
4. **帰納バイアスのスペクトル**

## 目次

1. [CNNの限界の振り返り](#section1)
2. [Vision Transformer](#section2)
3. [MLP-Mixer](#section3)
4. [帰納バイアスの比較](#section4)
5. [まとめ](#summary)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle, FancyBboxPatch, FancyArrowPatch
import japanize_matplotlib

plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12

<a id="section1"></a>
## 1. CNNの限界の振り返り

CNNの主な制約：
1. **局所的な受容野**：長距離依存関係を捉えにくい
2. **固定的な帰納バイアス**：平行移動のみ等変
3. **階層的な処理**：大域的な情報は深い層でのみ利用可能

In [ ]:
def visualize_architecture_evolution():
    """アーキテクチャの進化を可視化"""
    fig, ax = plt.subplots(figsize=(16, 8))
    
    # タイムライン
    architectures = [
        (2012, 'AlexNet', 'CNN', 'lightblue'),
        (2014, 'VGGNet', 'CNN', 'lightblue'),
        (2015, 'ResNet', 'CNN', 'lightblue'),
        (2017, 'Transformer\n(NLP)', 'Attention', 'lightyellow'),
        (2020, 'ViT', 'Attention', 'lightgreen'),
        (2021, 'MLP-Mixer', 'MLP', 'lightcoral'),
        (2022, 'ConvNeXt', 'CNN+Modern', 'lightblue'),
    ]
    
    for year, name, arch_type, color in architectures:
        x = (year - 2012) / 10
        box = FancyBboxPatch((x - 0.04, 0.3), 0.08, 0.4,
                             boxstyle="round,pad=0.02",
                             facecolor=color, edgecolor='gray',
                             transform=ax.transAxes)
        ax.add_patch(box)
        ax.text(x, 0.75, name, ha='center', va='center', fontsize=10,
               transform=ax.transAxes, fontweight='bold')
        ax.text(x, 0.5, arch_type, ha='center', va='center', fontsize=9,
               transform=ax.transAxes)
        ax.text(x, 0.2, str(year), ha='center', va='center', fontsize=10,
               transform=ax.transAxes)
    
    # 凡例
    from matplotlib.patches import Patch
    legend_elements = [
        Patch(facecolor='lightblue', label='CNN系'),
        Patch(facecolor='lightyellow', label='Transformer (NLP)'),
        Patch(facecolor='lightgreen', label='Vision Transformer'),
        Patch(facecolor='lightcoral', label='MLP系'),
    ]
    ax.legend(handles=legend_elements, loc='upper left', fontsize=10)
    
    ax.set_xlim(0, 1)
    ax.set_ylim(0, 1)
    ax.axis('off')
    ax.set_title('画像認識アーキテクチャの進化', fontsize=16, fontweight='bold')
    
    plt.tight_layout()
    plt.show()

visualize_architecture_evolution()

<a id="section2"></a>
## 2. Vision Transformer (ViT)

### 基本アイデア

1. 画像を**パッチ**に分割
2. 各パッチを**トークン**として扱う
3. **Self-Attention**で全パッチ間の関係をモデリング

In [ ]:
def visualize_vit_architecture():
    """ViTのアーキテクチャを可視化"""
    fig, axes = plt.subplots(1, 4, figsize=(20, 5))
    
    # 1. 入力画像
    ax = axes[0]
    img = np.random.rand(8, 8)
    ax.imshow(img, cmap='viridis')
    # パッチ分割を表示
    for i in range(0, 9, 2):
        ax.axhline(y=i-0.5, color='red', linewidth=2)
        ax.axvline(x=i-0.5, color='red', linewidth=2)
    ax.set_title('1. 入力画像\n（パッチに分割）', fontsize=12)
    ax.axis('off')
    
    # 2. パッチのフラット化
    ax = axes[1]
    patches = []
    for i in range(4):
        for j in range(4):
            patch = img[i*2:(i+1)*2, j*2:(j+1)*2]
            patches.append(patch.flatten())
    patches = np.array(patches)
    
    ax.imshow(patches, cmap='viridis', aspect='auto')
    ax.set_xlabel('パッチ内の値')
    ax.set_ylabel('パッチ番号')
    ax.set_title('2. パッチをフラット化\n（16パッチ → 16トークン）', fontsize=12)
    
    # 3. Self-Attention
    ax = axes[2]
    # アテンションマップ（ランダム）
    np.random.seed(42)
    attention = np.random.rand(16, 16)
    attention = attention / attention.sum(axis=1, keepdims=True)
    
    ax.imshow(attention, cmap='Blues')
    ax.set_xlabel('Key パッチ')
    ax.set_ylabel('Query パッチ')
    ax.set_title('3. Self-Attention\n（全パッチ間の関係）', fontsize=12)
    
    # 4. 特徴の意味
    ax = axes[3]
    ax.text(0.5, 0.8, 'Self-Attentionの利点', fontsize=14, 
           ha='center', transform=ax.transAxes, fontweight='bold')
    
    benefits = [
        '• 長距離依存関係を直接モデル化',
        '• 大域的なコンテキストを即座に取得',
        '• 動的な特徴の重み付け',
        '• 入力依存の受容野',
    ]
    
    for i, benefit in enumerate(benefits):
        ax.text(0.1, 0.6 - i*0.15, benefit, fontsize=11, 
               transform=ax.transAxes)
    
    ax.axis('off')
    
    plt.suptitle('Vision Transformer (ViT) の仕組み', fontsize=16, fontweight='bold')
    plt.tight_layout()
    plt.show()

visualize_vit_architecture()

In [ ]:
def explain_self_attention():
    """Self-Attentionの説明"""
    print("="*60)
    print("Self-Attention の仕組み")
    print("="*60)
    
    print("""
【数式】
Attention(Q, K, V) = softmax(QK^T / √d_k) V

- Q (Query): 「何を探しているか」
- K (Key): 「何を持っているか」
- V (Value): 「実際の値」
- d_k: キーの次元（スケーリング用）

【直感的理解】
1. 各パッチが「どの他のパッチに注目すべきか」を学習
2. QK^T で類似度を計算
3. softmax で正規化（重みの合計=1）
4. 重み付き和で出力を計算

【CNNとの違い】
CNN:  各位置は固定された近傍のみを見る（局所的）
ViT:  各位置は全ての位置を見れる（大域的）
    """)

explain_self_attention()

In [ ]:
def visualize_cnn_vs_vit_receptive_field():
    """CNNとViTの受容野比較"""
    fig, axes = plt.subplots(1, 2, figsize=(14, 6))
    
    # CNN
    ax = axes[0]
    for i in range(8):
        for j in range(8):
            rect = Rectangle((j, 7-i), 0.9, 0.9, facecolor='white', edgecolor='gray')
            ax.add_patch(rect)
    
    # 中心のニューロンの受容野（3x3 × 2層 = 5x5）
    center = (3.5, 3.5)
    for i in range(2, 6):
        for j in range(2, 6):
            rect = Rectangle((j, 7-i), 0.9, 0.9, facecolor='lightblue', 
                             edgecolor='blue', alpha=0.5)
            ax.add_patch(rect)
    
    # 中心
    rect = Rectangle((3, 4), 0.9, 0.9, facecolor='red', edgecolor='red')
    ax.add_patch(rect)
    
    ax.set_xlim(-0.5, 8.5)
    ax.set_ylim(-0.5, 8.5)
    ax.set_aspect('equal')
    ax.set_title('CNN: 局所的な受容野\n（深い層で徐々に拡大）', fontsize=12)
    ax.axis('off')
    
    # ViT
    ax = axes[1]
    for i in range(8):
        for j in range(8):
            color = 'lightblue' if not (i == 3 and j == 3) else 'red'
            rect = Rectangle((j, 7-i), 0.9, 0.9, facecolor=color, 
                             edgecolor='blue' if color == 'lightblue' else 'red',
                             alpha=0.5 if color == 'lightblue' else 1)
            ax.add_patch(rect)
    
    ax.set_xlim(-0.5, 8.5)
    ax.set_ylim(-0.5, 8.5)
    ax.set_aspect('equal')
    ax.set_title('ViT: 大域的な受容野\n（最初の層から全体を見れる）', fontsize=12)
    ax.axis('off')
    
    plt.suptitle('受容野の比較：CNN vs ViT', fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()

visualize_cnn_vs_vit_receptive_field()

<a id="section3"></a>
## 3. MLP-Mixer

### 驚きの発見

畳み込みもAttentionも使わず、**MLPだけ**で画像認識が可能！

In [ ]:
def visualize_mlp_mixer():
    """MLP-Mixerの仕組み"""
    fig, axes = plt.subplots(1, 3, figsize=(18, 6))
    
    # 1. パッチの配列
    ax = axes[0]
    # パッチ×特徴 の行列
    data = np.random.rand(9, 6)  # 9パッチ、6特徴
    ax.imshow(data, cmap='viridis', aspect='auto')
    ax.set_xlabel('特徴次元')
    ax.set_ylabel('パッチ')
    ax.set_title('入力: パッチ × 特徴 行列', fontsize=12)
    
    # 2. Token Mixing
    ax = axes[1]
    ax.text(0.5, 0.85, 'Token Mixing MLP', fontsize=14, 
           ha='center', transform=ax.transAxes, fontweight='bold')
    
    ax.text(0.5, 0.65, '列方向（パッチ間）に\nMLPを適用', fontsize=12, 
           ha='center', transform=ax.transAxes)
    
    # 矢印で方向を示す
    ax.annotate('', xy=(0.3, 0.35), xytext=(0.3, 0.5),
               arrowprops=dict(arrowstyle='->', color='blue', lw=3),
               transform=ax.transAxes)
    ax.text(0.35, 0.42, 'パッチ間の\n情報交換', fontsize=10, transform=ax.transAxes)
    
    ax.text(0.5, 0.15, 'Channel Mixing MLP', fontsize=14, 
           ha='center', transform=ax.transAxes, fontweight='bold')
    
    ax.annotate('', xy=(0.7, 0.25), xytext=(0.55, 0.25),
               arrowprops=dict(arrowstyle='->', color='red', lw=3),
               transform=ax.transAxes)
    ax.text(0.6, 0.28, '特徴間の\n情報交換', fontsize=10, transform=ax.transAxes)
    
    ax.axis('off')
    ax.set_title('MLP-Mixer の2種類のMLP', fontsize=12)
    
    # 3. 比較表
    ax = axes[2]
    comparison = [
        ('', 'CNN', 'ViT', 'MLP-Mixer'),
        ('空間的混合', '畳み込み', 'Self-Att.', 'Token MLP'),
        ('チャンネル混合', '1×1 Conv', 'MLP', 'Channel MLP'),
        ('帰納バイアス', '強い', '弱い', '中程度'),
        ('長距離依存', '層で成長', '即座', '即座'),
    ]
    
    table = ax.table(cellText=comparison, loc='center', cellLoc='center')
    table.auto_set_font_size(False)
    table.set_fontsize(10)
    table.scale(1.2, 1.8)
    
    # ヘッダーを太字に
    for i in range(4):
        table[(0, i)].set_text_props(fontweight='bold')
    
    ax.axis('off')
    ax.set_title('アーキテクチャ比較', fontsize=12)
    
    plt.suptitle('MLP-Mixer: 畳み込みもAttentionも使わない', fontsize=16, fontweight='bold')
    plt.tight_layout()
    plt.show()

visualize_mlp_mixer()

<a id="section4"></a>
## 4. 帰納バイアスの比較

各アーキテクチャは異なる帰納バイアスを持ちます。

In [ ]:
def visualize_inductive_bias_spectrum():
    """帰納バイアスのスペクトル"""
    fig, ax = plt.subplots(figsize=(14, 8))
    
    # スペクトル（帰納バイアスの強さ）
    architectures = [
        ('MLP\n(全結合)', 0.1, 'lightcoral'),
        ('ViT\n(パッチ分割のみ)', 0.3, 'lightgreen'),
        ('MLP-Mixer\n(パッチ + 位置)', 0.4, 'lightyellow'),
        ('CNN\n(局所性 + 重み共有)', 0.7, 'lightblue'),
        ('G-CNN\n(回転等変)', 0.85, 'lavender'),
    ]
    
    for name, bias, color in architectures:
        ax.barh(name, bias, color=color, edgecolor='gray', height=0.6)
        ax.text(bias + 0.02, name, f'{bias:.1f}', va='center', fontsize=11)
    
    ax.set_xlabel('帰納バイアスの強さ', fontsize=12)
    ax.set_xlim(0, 1)
    ax.set_title('アーキテクチャ別の帰納バイアス', fontsize=14, fontweight='bold')
    
    # 注釈
    ax.axvline(x=0.5, color='gray', linestyle='--', alpha=0.5)
    ax.text(0.25, -0.8, '← 弱い帰納バイアス\n（より多くのデータが必要）', 
           ha='center', fontsize=10, transform=ax.transData)
    ax.text(0.75, -0.8, '強い帰納バイアス →\n（少ないデータで学習可能）', 
           ha='center', fontsize=10, transform=ax.transData)
    
    plt.tight_layout()
    plt.show()

visualize_inductive_bias_spectrum()

In [ ]:
def compare_data_efficiency():
    """データ効率の比較"""
    fig, ax = plt.subplots(figsize=(12, 6))
    
    data_sizes = np.logspace(4, 8, 50)  # 10K to 100M
    
    # 仮想的な性能曲線
    cnn_perf = 0.9 * (1 - np.exp(-data_sizes / 1e6))
    vit_perf = 0.95 * (1 - np.exp(-data_sizes / 1e7))
    mlp_perf = 0.85 * (1 - np.exp(-data_sizes / 5e7))
    
    ax.semilogx(data_sizes, cnn_perf, 'b-', label='CNN', linewidth=2)
    ax.semilogx(data_sizes, vit_perf, 'g-', label='ViT', linewidth=2)
    ax.semilogx(data_sizes, mlp_perf, 'r-', label='MLP', linewidth=2)
    
    # 交差点を強調
    ax.axvline(x=1e7, color='gray', linestyle='--', alpha=0.5)
    ax.text(1e7, 0.3, 'CNNとViTが\n交差する領域', fontsize=10, ha='center')
    
    ax.set_xlabel('訓練データ数', fontsize=12)
    ax.set_ylabel('性能', fontsize=12)
    ax.set_title('データ量と性能の関係（概念図）', fontsize=14, fontweight='bold')
    ax.legend(fontsize=11)
    ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    print("観察:")
    print("- 少ないデータ: 強い帰納バイアスを持つCNNが有利")
    print("- 大量データ: 弱い帰納バイアスのViTが最終的に高性能")
    print("- トレードオフ: 帰納バイアスの強さ vs データ効率 vs 最終性能")

compare_data_efficiency()

<a id="summary"></a>
## 5. まとめ

### アーキテクチャの選択指針

| 状況 | 推奨アーキテクチャ |
|------|------------------|
| 少ないデータ | CNN |
| 大量のデータ | ViT |
| 長距離依存が重要 | ViT / MLP-Mixer |
| 回転不変性が必要 | G-CNN / データ拡張 |
| 推論速度重視 | CNN / MLP-Mixer |

### 学んだこと

1. **ViT**: Self-Attentionで大域的な依存関係をモデル化
2. **MLP-Mixer**: 畳み込みもAttentionも不要という驚きの発見
3. **帰納バイアスはトレードオフ**: 強すぎても弱すぎても問題
4. **問題とデータに応じた選択**が重要

### Section Cの終わり

これでSection C（帰納バイアスの科学）は終了です。

次のSection Dでは、**空間知性の応用**（セマンティックセグメンテーション、U-Netなど）について学びます。